<a href="https://colab.research.google.com/github/jpabloglez/Master_IA_Sanidad/blob/main/2_3_3_Covid19_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import psycopg2 # PostgreSQL database adapter -> https://pypi.org/project/psycopg2/
import pandas as pd
from pandas.plotting import register_matplotlib_converters 
register_matplotlib_converters() # Nos permite adaptar los formatos utilizados por pandas para definir datos del tipo fechas (y otros) a la definición 
# empleada por matplotlib
import matplotlib.pyplot as plt

# Conectamos con covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19',
    user='covid19',
    password='covid19'
)

# Solicitamos la información que nos interesa aplicando una consulta (query) de SQL
cur = conn.cursor()
cur.execute("""SELECT i.* FROM
                (SELECT max(date) AS date, countrycode, adm_area_1
                FROM epidemiology WHERE source=%(source)s
                GROUP BY countrycode, adm_area_1 ) AS recent_data
              INNER JOIN epidemiology AS i
              ON i.date=recent_data.date 
              AND i.countrycode = recent_data.countrycode
              AND i.adm_area_1 = recent_data.adm_area_1
              AND i.adm_area_1 <> ''
              AND (i.adm_area_2 IS NULL OR i.adm_area_2 = '')
              AND i.source=%(source)s
              ORDER by i.confirmed DESC
            """, {'source': 'ESP_MS'})
# Ver este enlace para acceder a las fuentes de los datos -> https://github.com/covid19db/data

covid19_data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

In [17]:
print("Primeras filas de la tabla:\n", covid19_data.head())
print("Resumen de estadísiticas:\n", covid19_data.describe())

Primeras filas de la tabla:
    source        date country  ... hospitalised_icu quarantined         gid
0  ESP_MS  2021-08-26   Spain  ...             None        None   [ESP.6_1]
1  ESP_MS  2021-08-26   Spain  ...             None        None   [ESP.8_1]
2  ESP_MS  2021-08-27   Spain  ...             None        None   [ESP.1_1]
3  ESP_MS  2021-08-26   Spain  ...             None        None  [ESP.10_1]
4  ESP_MS  2021-08-26   Spain  ...             None        None   [ESP.5_1]

[5 rows x 15 columns]
Resumen de estadísiticas:
            confirmed          dead
count      19.000000     19.000000
mean   253918.157895   4415.789474
std    288820.816005   4875.519341
min      7159.000000    101.000000
25%     75546.500000    919.000000
50%    136680.000000   2050.000000
75%    272795.000000   6672.500000
max    887826.000000  15780.000000
